In [11]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url='https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_73ee5d4ef51c4a3a813594efdd441667.csv'
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1


In [2]:
df.shape

(270, 14)

In [3]:
df=df.rename(columns={13:"Class"})

y=df[["Class"]]

y.groupby("Class")["Class"].count()

Class
1    150
2    120
Name: Class, dtype: int64

In [4]:
X=df.drop(["Class"],axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3


In [61]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")

fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
12,101.985020
11,70.098369
8,57.169619
7,56.909111
9,56.729134
2,56.554610
10,34.477790
1,26.065252
0,12.651991
6,9.190828


In [62]:
X_scores=scores[scores["scores"]>20]
len(X_scores)

8

In [63]:
Xp=X.iloc[:,X_scores.index]
Xp

,12,11,8,7,9,2,10,1
0,3,3,0,109,2.4,4,2,1
1,7,0,0,160,1.6,3,2,0
2,7,0,0,141,0.3,2,1,1
3,7,1,1,105,0.2,4,2,1
4,3,1,1,121,0.2,2,1,0
...,...,...,...,...,...,...,...,...
265,7,0,0,162,0.5,3,1,1
266,7,0,0,173,0.0,2,1,1
267,3,0,0,153,1.3,2,2,0
268,6,0,0,148,0.4,4,2,1


In [64]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
Xp=pd.DataFrame(scaler.fit_transform(Xp))
Xp.head()

,0,1,2,3,4,5,6,7
0,-0.875706,2.472682,-0.701222,-1.759208,1.181012,0.870928,0.676419,0.689500
1,1.189277,-0.711535,-0.701222,0.446409,0.481153,-0.183559,0.676419,-1.450327
2,1.189277,-0.711535,-0.701222,-0.375291,-0.656118,-1.238045,-0.954234,0.689500
3,1.189277,0.349871,1.426081,-1.932198,-0.743600,0.870928,0.676419,0.689500
4,-0.875706,0.349871,1.426081,-1.240239,-0.743600,-1.238045,-0.954234,-1.450327


In [65]:
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [68]:
BForest=BalancedRandomForestClassifier(n_estimators=270,min_samples_split=4,
                                       random_state=0)
scores=cross_val_score(BForest,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8482221246707637 +/- 0.030066832346999892


In [69]:
y_pred = cross_val_predict(BForest, Xp,np.asarray(y).reshape(y.shape[0],), cv=4)
AUC = auc(y, y_pred)
print("AUC for the prediction of the first disease: ",AUC)

AUC for the prediction of the first disease:  0.8441666666666666


In [70]:
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[132  18]
 [ 23  97]]


In [85]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.naive_bayes import GaussianNB
BBagging=BalancedBaggingClassifier(base_estimator=GaussianNB(),n_estimators=12,sampling_strategy="not majority",
                                   random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8444359086918349 +/- 0.007497104372390253


In [86]:
NB=GaussianNB()
scores=cross_val_score(NB,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8444907813871817 +/- 0.012135180627769169


#### Wrapping "GaussianNB" inside "BalancedBaggingClassifier" resulted in a more stable model, as indicated by smaller standard deviation.

In [105]:
BBagging=BalancedBaggingClassifier(base_estimator=RandomForestClassifier(criterion="entropy",min_samples_leaf=2),
                                   n_estimators=11,
                                   sampling_strategy="not majority",
                                   random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8519534679543459 +/- 0.035675692929736005


In [189]:
BBagging=BalancedBaggingClassifier(base_estimator=svm.SVC(kernel="linear",random_state=0),n_estimators=12,
                                  sampling_strategy="not majority",random_state=0)
scores=cross_val_score(BBagging,Xp,np.asarray(y).reshape(y.shape[0],),cv=8)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.848150623885918 +/- 0.040215582485645766


In [108]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(Xp,y,test_size=0.33,random_state=42)
print(X_train.shape)
print(X_test.shape)

(180, 8)
(90, 8)


In [109]:
BForest=BalancedRandomForestClassifier(n_estimators=270,min_samples_split=4,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
BForest

BalancedRandomForestClassifier(min_samples_split=4, n_estimators=270,
                               random_state=0)

In [111]:
from sklearn.metrics import accuracy_score
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9611111111111111
Accuracy on test data:  0.8333333333333334


In [112]:
BBagging=BalancedBaggingClassifier(base_estimator=GaussianNB(),n_estimators=12,sampling_strategy="not majority",
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
BBagging

BalancedBaggingClassifier(base_estimator=GaussianNB(), n_estimators=12,
                          random_state=0, sampling_strategy='not majority')

In [113]:
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8555555555555555
Accuracy on test data:  0.8666666666666667


In [114]:
NB=GaussianNB().fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
NB

GaussianNB()

In [115]:
train_pred=NB.predict(X_train)
test_pred=NB.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8555555555555555
Accuracy on test data:  0.8444444444444444


#### Again, wrapping GaussianNB inside BalancedBaggingClassifier produced better results.

In [116]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [119]:
NB=GaussianNB().fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
NB

GaussianNB()

In [120]:
train_pred=NB.predict(X_res)
test_pred=NB.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8385416666666666
Accuracy on test data:  0.8333333333333334


#### Resampling the train data to balance class distribution resulted in worse model performance for the GaussianNB model.

In [123]:
from sklearn.naive_bayes import BernoulliNB
NB2=BernoulliNB().fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
NB2

BernoulliNB()

In [124]:
train_pred=NB2.predict(X_res)
test_pred=NB2.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8333333333333334
Accuracy on test data:  0.8111111111111111


In [125]:
NB2p=BernoulliNB().fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
NB2p

BernoulliNB()

In [127]:
train_pred=NB2p.predict(X_train)
test_pred=NB2p.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8666666666666667
Accuracy on test data:  0.8111111111111111


#### For BernoulliNB, resampling had not effect on model performance.

In [128]:
Forest=RandomForestClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
Forest

RandomForestClassifier(random_state=0)

In [129]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.8333333333333334


In [130]:
Forest2=RandomForestClassifier(random_state=0).fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
Forest2

RandomForestClassifier(random_state=0)

In [131]:
train_pred=Forest.predict(X_res)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9947916666666666
Accuracy on test data:  0.8333333333333334


#### For RandomForestClassifier, resampling had no impact on model performance.

In [158]:
from sklearn import svm
SVM=svm.SVC(kernel="linear",random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
SVM

SVC(kernel='linear', random_state=0)

In [159]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.85
Accuracy on test data:  0.8666666666666667


In [160]:
SVM=svm.SVC(kernel="linear",random_state=0).fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
SVM

SVC(kernel='linear', random_state=0)

In [161]:
train_pred=SVM.predict(X_res)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8645833333333334
Accuracy on test data:  0.8333333333333334


#### For the Support Vector Machine model, resampling the train data resulted in worse model performance.

In [164]:
from sklearn.ensemble import GradientBoostingClassifier
Gradient=GradientBoostingClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],) )
Gradient

GradientBoostingClassifier(random_state=0)

In [165]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9888888888888889
Accuracy on test data:  0.7555555555555555


In [166]:
Gradient=GradientBoostingClassifier(random_state=0).fit(X_res,np.asarray(y_res).reshape(y_res.shape[0],) )
Gradient

GradientBoostingClassifier(random_state=0)

In [167]:
train_pred=Gradient.predict(X_res)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_res,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.9895833333333334
Accuracy on test data:  0.8222222222222222


#### Resampling the train data improved GradientBoostingClassifier's performance on test data. However, it is not the best result so far.

### Summary: Best models (All models used only 8 of the 13 predictor variables.)

#### A. BalancedRandomForestClassifier with 4-fold cross validation: 84.8% +/-  3%.

#### B. BalancedBaggingClassifier with GaussianNB as base estimator, 4-fold cross validation: 84.4% +/- 0.07% (most stable cross validation model)

#### C.BalancedBaggingClassifier with SVM as base estimator and "linear" kernel, 8-fold cross validation: 84.8% +/- 4%

#### D. BalancedBaggingClassifer with RandomForestClassifer as base estimator, 4-fold cross validation: 85.2% +/- 3.6%

#### E.BalancedBaggingClassifer with GaussianNB as base estimator, split the data into 67% for training and 33% for testing, no resampling: 86.7% accuracy on test data.

#### The most promising model of all is the GaussianNB model. With cross validation, GaussianNB by itself produced an accuracy of 84.4% as well. However, standard deviation was 1.2%. Wrapping GaussianNB inside BalancedBaggingClassifier reduced the standard deviation to 0.07%, a more stable model. With train_test_split, the impact of wrapping GaussianNB inside BalancedBaggingClassifier was even larger. With train_test_split, GaussianNB by itself had a 84.4% accuracy on test data. By wrapping GaussianNB inside BalancedBaggingClassifier, accuracy on test data was improved to 86.7%.

#### Resamping did not improve model performance on this dataset.